### Русский текст

Вчера был очень необычный день. Сначала я услышал странный разговор. Один говорил другому: «Это невозможно! Эти типы стали есть на складе! Мало того, что после них образовалась течь, так мне потом пришлось и печь чистить!». Этот разговор я слышал, пока шел дорогой домой: видимо, они работают на заводе (у одного из них была в руках пила). Потом, когда я проходил мимо парка, я увидел, что от него остались только пни, а вокруг них жгут траву. Видимо, это они спугнули целый рой пчел, и он полетел за мной. Было страшно, как будто бы к виску приставили дуло пистолета. В общем, день был такой неординарный, что я даже решил стих о нем сочинить.

В тексте присутствуют неоднозначные формы: "стали", "течь", "печь", "дорогой", "пила", "пни", "жгут", "рой", "дуло", "стих" и другие.

### Английский текст

Yesterday was a really bad day. Firstly, I overheard a strange talk. One person was speaking to the other: “That is unbearable! There was a fellow who started eating at the warehouse! Not only I had to fix a leak after them, but I also had to clean the stove!”. I heard this conversation while I was on my way home. It seemed like both of them work at the factory (one of them had a saw in his hands). Later, when I was going past the park, I saw that only stumps were left of it and they were burning the grass around them. Apparently, they have scared off a swarm of bees and it started chasing me. I was so scared as if someone was threatening me with a gun. All in all, the day was so unusual that I have decided to write a few rhymes about it.

Для английского текста даже не нужно было подбирать неоднозначные слова: они сами собой появились в переводе русского текста. По большей части это омонимия глагола и существительного-результата: "talk", "fix", "leak", "gun" и другие. Присутствуют также другие типы омонимий: прилагательное/существительное ("fellow"), несвязанные по смыслу существительное и форма глагола ("saw").

Я решил добавить еще один тэггер для русского -- rnnmorph. Он создан на основе pymorphy2, и его преимущество в том, что он учитывает контекст.

In [19]:
import glob

import nltk
from nltk import word_tokenize, sent_tokenize

import pandas as pd

from pymystem3 import Mystem
m = Mystem()

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

from sklearn.metrics import accuracy_score

import spacy
nlp = spacy.load(r"C:\Users\User\AppData\Roaming\Python\Python37\site-packages\en_core_web_sm\en_core_web_sm-2.3.1")

from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load('upos')

import re

2020-10-22 01:20:46,000 loading file C:\Users\User\.flair\models\en-pos-ontonotes-v0.4.pt


## Русский язык

In [36]:
text_paths = glob.glob('*.txt')

for i in text_paths:
    with open(i, encoding='utf-8') as f:
        text = f.read()
        text = [w for w in word_tokenize(text) if w.isalpha()]
        df = pd.DataFrame(text, columns={'token'})
        df.to_csv(i.replace('txt', 'csv'), sep='\t')        

In [12]:
russian_df = pd.read_csv('russian.csv', sep='\t', index_col=0)
english_df = pd.read_csv('english.csv', sep='\t', index_col=0)

with open('russian.txt', encoding='utf-8') as f:
    russian = f.read()
with open('english.txt', encoding='utf-8') as f:
    english = f.read()

In [30]:
russian_df.to_csv('russian.csv', sep='\t')
english_df.to_csv('english.csv', sep='\t')

In [62]:
pymorphy_pred = []

for i in russian_df['token']:
    ana = morph.parse(i)[0]
    pymorphy_pred.append(ana.tag.POS)
    
russian_df['pymorphy_pred'] = pd.Series(pymorphy_pred)

In [69]:
mystem_pred = []

ana = m.analyze(russian)
for word in ana:
    if 'analysis' in word:
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        mystem_pred.append(pos)

russian_df['mystem_pred'] = pd.Series(mystem_pred)

In [57]:
doc = Doc(russian)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

natasha_pred = []

for i in doc.sents:
    for j in i.morph.tokens:
        if j.pos != 'PUNCT':
            natasha_pred.append(j.pos)

russian_df['natasha_pred'] = pd.Series(natasha_pred)

In [64]:
forms = predictor.predict(list(russian_df['token']))
rnnmorph_pred = [ana.pos for ana in forms]

russian_df['rnnmorph_pred'] = pd.Series(rnnmorph_pred)

In [101]:
def convert_russian_tag(tag):
    if tag in [
        'ADVB',
        'ADV',
        'ADVPRO'
    ]:
        standard_tag = 'ADV'
    elif tag in [
        'VERB',
        'V',
        'AUX'
    ]:
        standard_tag = 'V'
    elif tag in [
        'ADJ',
        'ADJF',
        'A',
        'ADJS'
    ]:
        standard_tag = 'ADJ'
    elif tag in [
        'N',
        'NOUN',
        'S',
        'PROPN'
    ]:
        standard_tag = 'N'
    elif tag in [
        'PRO',
        'NPRO',
        'SPRO',
        'PRON',
        'APRO',
        'DET'
    ]:
        standard_tag = 'PRO'
    elif tag in [
        'NUM'
    ]:
        standard_tag = 'NUM'
    elif tag in [
        'CONJ',
        'SCONJ',
        'CCONJ'
    ]:
        standard_tag = 'CONJ'
    elif tag in [
        'VERB',
        'V',
        'INFN'
    ]:
        standard_tag = 'V'
    elif tag in [
        'PRCL',
        'PART'
    ]:
        standard_tag = 'PRCL'
    elif tag in [
        'INTJ'
    ]:
        standard_tag = 'INTJ'
    elif tag in [
        'PREP',
        'PR',
        'ADP'
    ]:
        standard_tag = 'PREP'
        
    return standard_tag

In [102]:
for i, j in russian_df.drop(['token', 'golden_pos'], axis=1).iterrows():
    for n in j:
        converted_tag = convert_russian_tag(n)
        if converted_tag == 'Error':
            print(converted_tag, n)

In [108]:
for i in ['pymorphy_pred', 'mystem_pred', 'natasha_pred', 'rnnmorph_pred']:
    russian_df[i] = russian_df[i].apply(convert_russian_tag)

In [105]:
russian_df

,token,golden_pos,pymorphy_pred,mystem_pred,natasha_pred,rnnmorph_pred
0,Вчера,ADV,ADV,ADV,ADV,ADV
1,был,V,V,V,V,V
2,очень,ADV,ADV,ADV,ADV,ADV
3,необычный,ADJ,A,A,A,A
4,день,N,N,N,N,N
...,...,...,...,...,...,...
106,решил,V,V,V,V,V
107,стих,N,V,N,N,N
108,о,PREP,PREP,PREP,PREP,PREP
109,нем,PRO,A,PRO,PRO,PRO


In [127]:
print('Accuracy scores for russian:')
print('')
for i in ['pymorphy', 'mystem', 'natasha', 'rnnmorph']:
    accuracy = accuracy_score(russian_df['golden_pos'], russian_df[i + '_pred'])
    print(f'{i}: {accuracy}')

Accuracy scores for russian:

pymorphy: 0.8108108108108109
mystem: 0.9009009009009009
natasha: 0.8738738738738738
rnnmorph: 0.8828828828828829


## Английский язык

In [9]:
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [136]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


True

In [142]:
nltk_pred = [ana[1]for ana in nltk.pos_tag(english_df['token'], tagset='universal')]

english_df['nltk_pred'] = pd.Series(nltk_pred)

In [164]:
spacy_doc = nlp(english)
spacy_pred = [token.pos_ for token in spacy_doc if token.pos_ != 'PUNCT']
english_df['spacy_pred'] = pd.Series(spacy_pred)

In [27]:
def convert_english_tag(tag):
    if tag in [
        'CCONJ',
        'CONJ'
    ]:
        converted_tag = 'CONJ'
    if tag in [
        'PART',
        'PRT'
    ]:
        converted_tag = 'PRT'
    else:
        converted_tag = tag
    
    return converted_tag

In [4]:
english_df['spacy_pred'] = english_df['spacy_pred'].apply(convert_english_tag)

In [28]:
english_sentences = sent_tokenize(english)
flair_pred = []
for i in english_sentences:
    sentence = Sentence(i)
    tagger.predict(sentence)
    flair_ana = sentence.to_tagged_string()
    flair_ana = [t for t in re.findall(r'<(.+?)>', flair_ana)]
    flair_ana = [t for t in flair_ana if t != 'PUNCT']
    flair_pred.extend(flair_ana)
    
flair_pred = [convert_english_tag(t) for t in flair_pred]
english_df['flair_pred'] = pd.Series(flair_pred)

In [29]:
english_df

,token,golden_pos,nltk_pred,spacy_pred,flair_pred
0,Yesterday,NOUN,NOUN,NOUN,NOUN
1,was,VERB,VERB,VERB,VERB
2,a,DET,DET,DET,DET
3,really,ADJ,ADV,ADV,ADV
4,bad,ADV,ADJ,ADJ,ADJ
...,...,...,...,...,...
146,a,DET,DET,DET,DET
147,few,ADJ,ADJ,ADJ,ADJ
148,rhymes,NOUN,NOUN,NOUN,NOUN
149,about,ADP,ADP,ADP,ADP


In [33]:
print('Accuracy scores for english:')
print('')
for i in ['nltk', 'spacy', 'flair']:
    accuracy = accuracy_score(english_df['golden_pos'], english_df[i + '_pred'])
    print(f'{i}: {accuracy}')

Accuracy scores for english:

nltk: 0.8741721854304636
spacy: 0.8609271523178808
flair: 0.8741721854304636
